<a href="https://colab.research.google.com/github/Shezan57/Computer-Vision/blob/main/digit_recog_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Required Libraries

In [3]:
!pip install torch torchvision

In [6]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

Downloading training and testing dataset

In [8]:
training_data = datasets.MNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor()
)

test_data = datasets.MNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor()
)

batch_size = 64

train_dataloader = DataLoader(training_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

for X, y in test_dataloader:
  print(f"shape of X [N, C, H, W]: {X.shape}")
  print(f"shape of y: {y.shape} {y.dtype}")

shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
shape of y: torch.Size([64]) torch.int64
shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
shape of y: torch.Size([64]) torch.int64
shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
shape of y: torch.Size([64]) torch.int64
shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
shape of y: torch.Size([64]) torch.int64
shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
shape of y: torch.Size([64]) torch.int64
shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
shape of y: torch.Size([64]) torch.int64
shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
shape of y: torch.Size([64]) torch.int64
shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
shape of y: torch.Size([64]) torch.int64
shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
shape of y: torch.Size([64]) torch.int64
shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
shape of y: torch.Size([64]) torch.int64
shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
shape o

Define Model

In [11]:
device = torch.device("cuda" if torch.cuda.is_available()
            else "mps" if torch.backends.mps.is_available()
            else "cpu")
print(f"Using {device} device")

class NeuralNetwork(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(input_size, hidden_size),
        nn.ReLU(),
        nn.Linear(hidden_size, hidden_size),
        nn.ReLU(),
        nn.Linear(hidden_size, num_classes)
    )

  def forward(self, image_tensor):
    image_tensor = self.flatten(image_tensor)
    logits = self.linear_relu_stack(image_tensor)
    return logits

input_size = 28*28
hidden_size = 512
num_classes = 10

model = NeuralNetwork(input_size, hidden_size, num_classes).to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Training loop

In [19]:
learning_rate = 1e-3
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()

  for batch_num, (X,y) in enumerate(dataloader):
    #forward pass to compute prediction
    pred = model(X)
    #compute prediction error using loss function
    loss = loss_fn(pred,y)
    #backword pass
    optimizer.zero_grad() #zero any previous gradient calculations
    loss.backward() #calculate gradient
    optimizer.step() # update model parameters

    if batch_num > 0 and batch_num % 100 == 0:
      loss, current = loss.item(), batch_num * len(X)
      print(f"loss: {loss: >7f} [{current: >5d} / {size: >5d}]")

Test Loop

In [13]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  for X, y in dataloader:
    pred = model(X)
    test_loss += loss_fn(pred, y).item()
    correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Train the model

In [20]:
epochs = 5
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.176398 [ 6400 / 60000]
loss: 0.137492 [12800 / 60000]
loss: 0.257808 [19200 / 60000]
loss: 0.111829 [25600 / 60000]
loss: 0.260043 [32000 / 60000]
loss: 0.132478 [38400 / 60000]
loss: 0.198450 [44800 / 60000]
loss: 0.206693 [51200 / 60000]
loss: 0.156863 [57600 / 60000]
Test Error: 
 Accuracy: 95.6%, Avg loss: 0.135500 

Epoch 2
-------------------------------
loss: 0.084980 [ 6400 / 60000]
loss: 0.103053 [12800 / 60000]
loss: 0.121451 [19200 / 60000]
loss: 0.054855 [25600 / 60000]
loss: 0.162521 [32000 / 60000]
loss: 0.059188 [38400 / 60000]
loss: 0.125197 [44800 / 60000]
loss: 0.189412 [51200 / 60000]
loss: 0.080632 [57600 / 60000]
Test Error: 
 Accuracy: 97.2%, Avg loss: 0.094124 

Epoch 3
-------------------------------
loss: 0.075788 [ 6400 / 60000]
loss: 0.080690 [12800 / 60000]
loss: 0.100872 [19200 / 60000]
loss: 0.019789 [25600 / 60000]
loss: 0.073433 [32000 / 60000]
loss: 0.021682 [38400 / 60000]
loss: 0.079209 [44800 / 60000]
l

Save the model and make predictions

In [21]:
# save our model parameters
torch.save(model.state_dict(), "ml_with_pytorch_model.pth")
print("Saved Pytorch Model State t0 ml_with_pytorch_model.pth")

#Load the saved model parameters into a new instance of the model
model = NeuralNetwork(input_size, hidden_size, num_classes)
model.load_state_dict(torch.load("ml_with_pytorch_model.pth"))

#inference using the new model instance
model.eval()
for i in range(10):
  x,y = test_data[i][0], test_data[i][1]

  x = x.to(device)
  pred = model(x)
  predicted, actual = pred[0].argmax(0), y
  print(f"predicted: {predicted}, actual: {actual}")

Saved Pytorch Model State t0 ml_with_pytorch_model.pth
predicted: 7, actual: 7
predicted: 2, actual: 2
predicted: 1, actual: 1
predicted: 0, actual: 0
predicted: 4, actual: 4
predicted: 1, actual: 1
predicted: 4, actual: 4
predicted: 9, actual: 9
predicted: 5, actual: 5
predicted: 9, actual: 9
